In [ ]:
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit
from scipy.integrate import simpson
from sklearn.linear_model import Ridge
import numpy as np
import matplotlib.pyplot as plt
import tkinter.filedialog as tkf
import hyperspy.api as hys
import tifffile
from scipy import ndimage
import ipywidgets as pyw
plt.rcParams['font.family'] = 'Times New Roman'
color_rep = ["black", "orange", "purple", "blue", "red", "green", "yellow", "lime", 
             "cyan", "magenta", "lightgray", "peru", "springgreen", "deepskyblue", 
             "hotpink", "darkgray"]

In [ ]:
s = [0,0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.5,0.6,0.7,0.8,0.9,1,1.2,1.4,1.6,1.8,2,2.5,3,4,6]
B_AF = [4.420007125,4.17476802,3.562240581,2.837929736,2.182057711,1.661922679,1.274102699,0.99236289,0.788187077,0.528119561,0.379835451,0.288583691,0.2281294,0.187066108,0.155127992,0.1140647,0.086689172,0.06843882,0.054751056,0.04562588,0.029656822,0.020531646,0.01140647,0.005703235]
C_AF = [3.921544386,3.761853806,3.35350218,2.827663913,2.30410694,1.84784814,1.477137865,1.18627288,0.961565421,0.655872025,0.471087211,0.354741217,0.277177221,0.224707459,0.185925461,0.134596346,0.101517583,0.07984529,0.065016879,0.053610409,0.035360057,0.025094234,0.013687764,0.006843882]
N_AF = [3.497223702,3.38772159,3.103700487,2.718161801,2.308669528,1.925412136,1.593483859,1.317447285,1.092739826,0.767655431,0.55891703,0.42203939,0.329646983,0.264630104,0.217863577,0.156268639,0.117486641,0.092392407,0.074142055,0.061594938,0.041063292,0.028516175,0.015969058,0.006843882]
O_AF = [3.14818572,3.070621724,2.865305264,2.576721573,2.255059119,1.937959253,1.649375562,1.397292575,1.185132233,0.860047838,0.63876232,0.487056269,0.382116745,0.306834043,0.252082987,0.179081579,0.134596346,0.104939524,0.084407878,0.069579467,0.04562588,0.033078763,0.018250352,0.007984529]

C_at = np.array([96.50306,97.0115,97.11562,97.078285,97.484886,97.40846,97.29728,97.188,97.19592,93.06823,93.21894,93.18482,93.07648,92.61424,90.72497,87.01168,82.85428,78.24188])
B_at = np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
N_at = np.array([0,0,0,0,0,0,0,0,0,0,0,0.02188113,0.22326997,1.3212804,3.341334,5.9228334,9.115244,10.623713])
O_at = np.array([3.4969392,2.988503,2.8843787,2.9217155,2.5151057,2.5915408,2.7027304,2.8119998,2.804078,6.9317694,6.7810555,6.793295,6.7002487,6.064484,5.9336953,7.0654817,8.030481,11.134399])
depth = np.arange(len(C_at))

In [ ]:
damp = tifffile.imread("./setting/filters/damping_filter.tif")
print(damp.shape)

filter_types = ["boxcar", "trangular", "trapezoidal", "Happ-Genzel", "3TEM BH", "4TEM BH"]

In [ ]:
B_AF_sqr = tifffile.imread("./setting/AFFs_300keV/05_Boron_300keV_fs_square_0.005A.tif").squeeze()
B_AF = np.sqrt(B_AF_sqr)
B_AF /= np.max(B_AF)
C_AF_sqr = tifffile.imread("./setting/AFFs_300keV/06_Carbon_300keV_fs_square_0.005A.tif").squeeze()
C_AF = np.sqrt(C_AF_sqr)
C_AF /= np.max(C_AF)
N_AF_sqr = tifffile.imread("./setting/AFFs_300keV/07_Nitrogen_300keV_fs_square_0.005A.tif").squeeze()
N_AF = np.sqrt(N_AF_sqr)
N_AF /= np.max(N_AF)
O_AF_sqr = tifffile.imread("./setting/AFFs_300keV/08_Oxygen_300keV_fs_square_0.005A.tif").squeeze()
O_AF = np.sqrt(O_AF_sqr)
O_AF /= np.max(O_AF)

k_scale = 0.005 # Angstrom
s = np.arange(0, k_scale*len(B_AF), k_scale)
print(len(s))

In [ ]:
raw_adr = tkf.askopenfilename()
print(raw_adr)

In [ ]:
# Load a data (DM)
rot_dp = hys.load(raw_adr)
print(rot_dp)
rot_dp_data = rot_dp.data.copy()
print(rot_dp_data.shape)
n_dim = len(rot_dp_data.shape)
print(n_dim)

In [ ]:
scale = []
origin = []
unit = []
size = []

if n_dim == 2:
    scale.append(rot_dp.axes_manager[0].scale)
    origin.append(rot_dp.axes_manager[0].offset)
    unit.append(rot_dp.axes_manager[0].units)
    size.append(1)   

for i in range(n_dim):
    print(rot_dp.axes_manager[i].scale, rot_dp.axes_manager[i].offset, rot_dp.axes_manager[i].units, rot_dp.axes_manager[i].size)
    scale.append(rot_dp.axes_manager[i].scale)
    origin.append(rot_dp.axes_manager[i].offset)
    unit.append(rot_dp.axes_manager[i].units)
    size.append(rot_dp.axes_manager[i].size)

In [ ]:
rot_dp_data = rot_dp_data / np.expand_dims(np.max(rot_dp_data, axis=n_dim-1), axis=-1)
if n_dim == 2:
    rot_dp_data = np.expand_dims(rot_dp_data, axis=0)
    n_dim = 3

In [ ]:
k_list = np.arange(origin[n_dim-1], origin[n_dim-1]+scale[n_dim-1]*size[n_dim-1], scale[n_dim-1])
print(k_list.shape)
print(k_list.min(), k_list.max())

In [ ]:
exp_depth = np.linspace(depth[0], depth[-1], rot_dp_data.shape[1])

interpolation = interp1d(depth, B_at*0.01, kind="cubic")
B_atp = interpolation(exp_depth)

interpolation = interp1d(depth, C_at*0.01, kind="cubic")
C_atp = interpolation(exp_depth)

interpolation = interp1d(depth, N_at*0.01, kind="cubic")
N_atp = interpolation(exp_depth)

interpolation = interp1d(depth, O_at*0.01, kind="cubic")
O_atp = interpolation(exp_depth)

In [ ]:
interpolation = interp1d(s, B_AF, kind="cubic")
NB_AF = interpolation(k_list)
NB_AF /= np.max(NB_AF)

interpolation = interp1d(s, C_AF, kind="cubic")
NC_AF = interpolation(k_list)
NC_AF /= np.max(NC_AF)

interpolation = interp1d(s, N_AF, kind="cubic")
NN_AF = interpolation(k_list)
NN_AF /= np.max(NN_AF)

interpolation = interp1d(s, O_AF, kind="cubic")
NO_AF = interpolation(k_list)
NO_AF /= np.max(NO_AF)

AF_mean = (NB_AF[np.newaxis, :]*B_atp[:, np.newaxis] + NC_AF[np.newaxis, :]*C_atp[:, np.newaxis] + 
           NN_AF[np.newaxis, :]*N_atp[:, np.newaxis] + NO_AF[np.newaxis, :]*O_atp[:, np.newaxis])
print(len(AF_mean))

AF_mean_square = AF_mean**2
print(len(AF_mean_square))

AF_square_mean = ((NB_AF[np.newaxis, :]*B_atp[:, np.newaxis])**2 + (NC_AF[np.newaxis, :]*C_atp[:, np.newaxis])**2 + 
                  (NN_AF[np.newaxis, :]*N_atp[:, np.newaxis])**2 + (NO_AF[np.newaxis, :]*O_atp[:, np.newaxis])**2)
print(len(AF_square_mean))

In [ ]:
%matplotlib inline

In [ ]:
%matplotlib widget

In [ ]:
%matplotlib qt

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
for i in range(len(AF_mean_square)):
    ax.plot(k_list, AF_mean_square[0], 'r-')
    ax.plot(k_list, AF_mean_square[-1], 'k-')
    ax.plot(k_list, AF_square_mean[0], 'g+')
    ax.plot(k_list, AF_square_mean[-1], 'b+')
ax.grid()
fig.tight_layout()
plt.show()

In [ ]:
def rif_to_rdf(rif, r_list, k_list):
    gr = []
    for i in range(len(r_list)):
        sin_rk = np.sin(2*np.pi*r_list[i]*k_list)
        rif_sin = rif * sin_rk
        gr_tmp = 8 * np.pi * simpson(rif_sin, dx=(k_list[1]-k_list[0]))
        gr.append(gr_tmp)
    
    return np.asarray(gr)

In [ ]:
r_size = 500
r_scale = 0.01 # Angstrom
r_list = np.arange(r_size) * r_scale
r_unit = unit[-1][-1]
print(r_list.shape)
print(r_scale)
print(r_unit)
#print(r_list)

In [ ]:
rgr_ridge = Ridge(alpha=0.0)

In [ ]:
%matplotlib widget
fig, ax = plt.subplots(2, 2, figsize=(8, 5))
int_img = np.sum(rot_dp_data, axis=2)

def RDF_4DSTEM(yp, xp, aph, fit_range, ln, ln_check, f_n, f_c, full_check):
    
    for a in ax.flat:
        a.cla()
        
    ax[0][0].imshow(int_img, cmap="gray")
    ax[0][0].axis("off")
    ax[0][0].scatter(xp, yp, c="red", s=15)
    
    filt = damp[f_n].copy()
    filt[int(f_c*len(filt)):] = 0
    print(filter_types[f_n])
    ind = np.linspace(0, len(filt)-1, size[n_dim-1]).astype(np.int16)
    filt = filt[ind]
    rot_dp_tmp = rot_dp_data[yp, xp]
    
    if ln_check:
        rgr_ridge.fit(AF_mean_square[xp, fit_range[0]:fit_range[1]].reshape(-1, 1), rot_dp_tmp[fit_range[0]:fit_range[1]].reshape(-1, 1))
        print("fitted N value = %f"%rgr_ridge.coef_[0][0])
        ln = rgr_ridge.coef_[0][0]
    
    Nfs_tmp = ln*AF_mean_square[xp]
    rif_tmp = ((rot_dp_tmp + aph - Nfs_tmp) / (ln*AF_square_mean[xp])) * k_list
    rif_tmp_filtered = (((rot_dp_tmp + aph - Nfs_tmp) / (ln*AF_square_mean[xp])) * k_list) * filt
    
    ax[0][1].plot(k_list, rot_dp_tmp, 'k-')
    ax[0][1].plot(k_list, Nfs_tmp, 'r-')
    ax[0][1].set_ylim(ymin=0.0, ymax=0.002)
    ax[0][1].fill_between([k_list[fit_range[0]], k_list[fit_range[1]]], np.max(rot_dp_tmp), alpha=0.5, color="orange")
    ax[0][1].grid()
    ax[1][0].plot(k_list, rif_tmp, 'k-')
    ax[1][0].plot(k_list, rif_tmp_filtered, 'g-')
    ax[1][0].fill_between([k_list[fit_range[0]], k_list[fit_range[1]]], np.max(rif_tmp), alpha=0.5, color="orange")
    ax[1][0].grid()
    
    tmp_Gr = rif_to_rdf(rif_tmp_filtered, r_list, k_list)
    ax[1][1].plot(r_list, tmp_Gr, 'k-')
    if not full_check:
        ax[1][1].set_xlim(xmin=1.0, xmax=2.5)
    ax[1][1].grid()
    
    fig.tight_layout()
    
st = {"description_width": "initial"}
y_wg = pyw.BoundedIntText(value=0, min=0, max=size[1]-1, description="y position", style=st)
x_wg = pyw.BoundedIntText(value=0, min=0, max=size[0]-1, description="x position", style=st)
N_wg = pyw.BoundedFloatText(value=1.0, min=0.00001, step=0.001, description="N", style=st)
N_manual = pyw.Checkbox(value=True, description="use the fitted value of N")
fit_range_wg = pyw.IntRangeSlider(value=[70, 80], 
                                  min=40, max=size[n_dim-1]-1, description="N fitting range")
alpha_wg = pyw.FloatText(value=0.0, step=0.000001, description="alpha", style=st)
filter_wg = pyw.BoundedIntText(value=0, min=0, max=len(filter_types)-1, description="filter", style=st)
filter_cut_wg = pyw.BoundedFloatText(value=0.8, min=0.0, max=1.0, step=0.05, description="filter high cut", style=st)
full_wg = pyw.Checkbox(value=True, description="show the full range of RDF")

pyw.interact(RDF_4DSTEM, yp=y_wg, xp=x_wg, aph=alpha_wg, fit_range=fit_range_wg, 
             ln=N_wg, ln_check=N_manual, f_n=filter_wg, f_c=filter_cut_wg, full_check=full_wg)
plt.show()

In [ ]:
alpha = alpha_wg.value
print(alpha)
N_check = N_manual.value
print(N_check)
N = N_wg.value
print(N)
filter_select = filter_wg.value
print(filter_select)
high_cut = filter_cut_wg.value
print(high_cut)
fit_range = fit_range_wg.value
print(fit_range)

In [ ]:
RIF_data = []
for i in range(size[1]):
    for j in range(size[0]):
        if N_check:
            rgr_ridge.fit(AF_mean_square[j, fit_range[0]:fit_range[1]].reshape(-1, 1),rot_dp_data[i, j][fit_range[0]:fit_range[1]].reshape(-1, 1))
            N = rgr_ridge.coef_[0][0]
        filt = damp[filter_select].copy()
        filt[int(high_cut*len(filt)):] = 0
        ind = np.linspace(0, len(filt)-1, size[n_dim-1]).astype(np.int16)
        filt = filt[ind]
        rif = (((rot_dp_data[i, j] + alpha - N*AF_mean_square[j]) / (N*AF_square_mean[j])) * k_list) * filt
        RIF_data.append(rif)
               
RIF_data = np.asarray(RIF_data).reshape(size[1], size[0], -1)

In [ ]:
%matplotlib inline

In [ ]:
%matplotlib widget

In [ ]:
%matplotlib qt

In [ ]:
Gr = []
for i in range(size[1]):
    for j in range(size[0]):
        tmp_Gr = rif_to_rdf(RIF_data[i, j], r_list, k_list)
        Gr.append(tmp_Gr)

Gr = np.asarray(Gr).reshape(size[1], size[0], -1)
print(Gr.shape)    

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
for i in range(size[0]):
    ax.plot(r_list, Gr[0, i])
ax.grid()
ax.set_xlim(xmin=1.0, xmax=3.0)
ax.set_ylim(ymin=-5.0, ymax=5.0)
fig.tight_layout()
plt.show()

In [ ]:
tifffile.imwrite(raw_adr[:-4]+"_Gr_upto_%dA_size_%d_04.tif"%(int(r_size*r_scale), r_size), Gr)

In [ ]:
tifffile.imwrite(raw_adr[:-4]+"_RIF.tif", RIF_data)